# 1 - Sequence to Sequence Learning with Neural Nertworks
- pytorch 와 torchtext를 활용해서 한 시퀀스에서 다른 시퀀스로 이동하는 머신러닝 모델을 구축할 것입니다.<br>
- 한글에서 영어로 번역할때 수행되지만 모델은 한 시퀀스에서 다른 시퀀스로 이동하는 것과 관련된 모든 문제에 적용될 수 있습니다. 즉, 동일한 언어의 시퀀스에서 더 짧은 시퀀스로 이동합니다.<br> 
- 첫 번째 노트북에서는 Neural Networks 논문의 모델을 구현하여 일반 개념을 이해하기 쉽게 할 것입니다.

## Introduction
- 가장 일반적인 시퀀스 대 시퀀스(seq2seq) 모델은 인코더-디코더 모델로 일반적으로 순환 신경망(RNN)을 사용하여 소스(입력) 문장을 단일 벡터로 인코딩합니다.
- 이 노트북에서는 이 단일 벡터를 context벡터라고 부를 것입니다. 컨텍스트 벡터는 전체 입력 문장의 추상적 표현으로 생각할 수 있습니다. 그런 다음 이 벡터는 한 번에 한 단어씩 생성하여 대상(출력)문장을 출력하는 방법을 학습하는 두 번째 RNN에 의해 디코딩됩니다.

<p align="center"><img src="https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq1.png"></p>

- 위 이미지는 translation예시를 보여줍니다. 입력/소스 문장 "guten morgen"은 임베딩 레이어(노란색)를 통과한 다음 인코더(녹색)에 입력됩니다. 또한 시퀀스 시작(`<sos>`)및 시퀀스 끝(`<eos>`)토큰을 문장의 시작과 끝에 각각 추가합니다.
- 각 시간 단계에서 인코더RNN에 대한 입력은 현재 단어 e(xt)의 임베딩 e와 이전 시간 단계 ht-1 및 인코더 RNN의 숨겨진 상태입니다. 새로운 은닉 상태 ht를 출력합니다. 지금까지는 은닉 상태를 문장의 백터표현으로 생각할 수 있습니다. RNN은 다음 두 가지의 함수로 나타낼 수 있습니다. $e(x_{t})$ 및 $h_{t-1}$:
$$ h_{t} = EncoderRNN(e(x_{t}),h_{t-1})$$
여기서는 일반적으로 RNN이라는 용어를 사용합니다. LSTM또는 GRU와 같은 모든 반복 아키텍처가 될 수 있습니다.
우리는 $ X=(x_{1},x_{2},...,x_{T})$, 여기서 $x_{1}=<sos>,x_{2}=guten$ 등 초기 은닉 상태, $h_{0}$ 는 일반적으로 0 또는 학습된 매개변수로 초기화됩니다.
마지막 단어, $x_{T}$ 는 임베딩 레이어를 통해 RNN으로 전달되었으며 최종 은닉 상태를 사용합니다. $h_{T}$,컨텍스트 벡터, 즉 $h_{T}=z$ 이것은 전체 소스 문장의 벡터 표현입니다.
이제 컨텍스트 벡터 $z$ 가 있으므로 출력/목표 문장 "좋은 아침"을 얻기 위해 디코딩을 시작할 수 있습니다. 다시 말하지만, 시퀀스 토큰의 시작과 끝을 대상 문장에 추가합니다. 각 시간 단계에서 디코더 RNN(파란색)에 대한 입력은 이전 시간 단계의 숨겨진 상태뿐만 아니라 현재 단어 현재 단어 ($y_{t}$)의 임베딩, d 및 이전 시간 단계 $s_{t-1}$ 의 숨겨진 상태이며, 여기서 초기 디코더 숨김 상태 $s_{0}$ 은 컨텍스트 벡터, 즉 초기 디코더 숨김 상태 $s_{0}=z$ 이다. 따라서 인코더와 유사하게 우리는 디코더를 다음과 같이 나타낼 수 있다.
$$ s_{t}=DecoderRNN(d(y_{t}),s_{t-1}) $$
입력/소스 임베딩 레이어 $e$ 와 출력/타겟 임베딩 레이어 $d$ 는 모두 다이어그램에서 노란색으로 표시되지만 고유한 매개변수가 있는 두 개의 서로 다른 임베딩 레이어입니다.

디코더에서 우리는 숨겨진 상태에서 실제 단어로 이동해야 하므로 각 시간 단계에서 $s_{t}$ 는(보라색으로 표시된 선형 레이어를 통과함으로써) 시퀀스의 다음 단어라고 생각하는 것을 예측합니다.
$$\hat y_{t} = f(s_{t})$$
디코더의 단어는 항상 시간 단계당 하나씩 차례로 생성됩니다. 디코더에 대한 첫 번째 입력인 $y_{1}$ 에 대해 항상 `<sos>` 를 사용하지만 후속 입력인 $y_{t>1} $ 에 대해서는 시퀀스의 실제 정답인 다음 단어인 $y_{t}$ 를 사용하고 때떄로 디코더에 의해 예측된 단어를 사용합니다. $y_{t}$ 를 사용하고 떄떄로 디코더에 의해 예측된 단어를 사용합니다.
$\hat y_{t-1}$ 이것을 교사 강제라고 합니다 자세한 내용은 [여기](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/)를 참고하세요.
우리의 모델을 훈련/테스트할 때 우리는 항상 목표 문장에 얼마나 많은 단어가 있는지 알고 잇으므로 일단 많이 맞으면 단어 생성을 멈춥니다. 추론하는 동안 모델이 `<eos>`토큰을 출력할 떄까지 또는 특정 양의 단어가 생성된 후 단어를 계속 생성하는 것이 일반적입니다.
목표 문장을 예측했다면, $\hat Y = \lbrace \hat y_{1}, \hat y_{2},...,\hat y_{r} \rbrace$, 실제 목표 문장과 비교합니다. $Y = \lbrace y_{1},y_{2},...,y_{T} \rbrace$, 손실을 계산합니다. 그런 다음 이 손실을 사용하여 모델의 모든 매개변수를 업데이트합니다.

## Preparing Data

### 필요 라이브러리 improt

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

C:\Users\INP016\anaconda3\envs\seq2seq_01\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- 고정된 결과를 위해서 임의의 시드를 고정합니다.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

- 다음으로 토크나이저를 생성합니다. 토크나이저는 문장을 포함하는 문자열을 해당 문자열을 구성하는 개별 토큰 목록으로 바꾸는 데 사용됩니다.
- 우리는 지금부터 문장이 단어의 연속이라고 말하는 대신 토큰의 연속이라는 것에 대해 이야기 할것입니다. 차이점은 "good" 과  "morning"은 단어이자 토큰이지만 "!"은 단어가 아니라 토큰입니다.
- spacy는 각 모델의 토크나이저에 엑세스할수 있도록 로드해야 하는 각 언어에 대한 모델이 있습니다. 여기서는 영어모델을 활용할 것입니다.
- 한글 형태소 분석은 mecab활용 할 것이며, 데이터셋은 [ai_hub](https://aihub.or.kr/)의 한영 병력코퍼스를 활용했습니다.

In [3]:
spacy_en = spacy.load('en_core_web_sm')

In [4]:
def tokenize_en(text):
  """
  Tokenizes English text from a string into a list of string (tokens)
  """
  return [tok.text for tok in spacy_en.tokenizer(text)]

### 한영 번역이므로 SRC:한글 -> TRG:영어로 설정

In [5]:
SRC = Field(tokenize = mecab.morphs, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = False)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

### Field 하이퍼파라미터
- sequential : 시퀀스 데이터 여부(True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부 (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정.(string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 자원을 맨 앞으로 하여 데이터를 불러올 것인지 여부 (False가 기본값)
- is_target : 레이블 데이터 여부 (False가 기본값)
- fix_length : 최대 허용 길이.이 길이에 맞춰서 패딩(padding)진행

In [6]:
df = pd.read_excel("data\kor_eng_corpus.xlsx")

In [7]:
len(df)

200000

In [8]:
train_df = df[:100000]
test_df = df[100000:]

In [9]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [10]:
from torchtext.legacy import data
from torchtext.legacy import datasets
#from torchtext.legacy import Iterator

In [40]:
train_data, valid_data, test_data = datasets.translation.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))




"\ntrain_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), \n                                                    fields = (SRC, TRG))\n\n\n"

In [64]:
train_data, valid_data = data.TabularDataset.splits(path = "", train='train_data.csv', test='test_data.csv', format='csv', fields=[('KOR',SRC), ('ENG',TRG)], skip_header=True)

In [65]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
#print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 100000
Number of validation examples: 100000


In [67]:
print(vars(train_data.examples[0]))

{'KOR': ['1'], 'ENG': ["'", 'bible', "coloring'은", '성경의', '아름다운', '이야기를', '체험', '할', '수', '있는', '컬러링', '앱입니다', '.']}


In [70]:
def custom_split(data):
  